<a href="https://colab.research.google.com/github/Pavan0604/IOT/blob/main/Optimized_Major.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy.lib.function_base import average
import pandas as pd
import matplotlib.pyplot as plt
import math
import plotly.express as px

In [ ]:
import itertools

class Node:
    def __init__(self):
        self.data = 0
        self.battery = 0
    def set_values(self,data,battery):
        self.data = data
        self.battery = battery
    def get_data(self):
        return self.data
    def get_battery(self):
        return self.battery

class Gateway:
    def __init__(self):
        self.power = 0
        self.storage = 0
    def set_values(self,power,storage):
        self.power = power
        self.storage = storage
    def get_power(self):
        return self.power
    def get_storage(self):
        return self.storage

class Graph:	## class for creating graph objects
    def __init__(self,num_of_nodes,num_of_gateways):
        self.rows=num_of_nodes
        self.cols=num_of_gateways
        self.adjmatrix=[]
        for i in range(self.rows):
            self.adjmatrix.append([0 for j in range(self.cols)])
    def set_values(self,row,col,value):
        self.adjmatrix[row][col]=value
    def print_graph(self):
        for i in range(self.rows):
            for j in range(self.cols):
                print(self.adjmatrix[i][j],end=" ")
            print('\n')
        print('\n')

def create_nodes(n): ## function to create nodes
    '''while True:
        try:
            n = int(input("Enter number of nodes: "))
        except ValueError:
                print("Invalid input, try again")
                continue
        if n<=0:
            print("Number of nodes cannot be negative & zero ")
            continue
        else:
            break'''
    nodes = [Node() for i in range(n)]
    for i in range(n):
        #battery = int(input("Enter the battery in node{}: ".format(i)))
        #data = int(input("Enter the data in node{}: ".format(i)))
        data  = np.random.randint(0,100)
        nodes[i].set_values(data,data)
    return nodes

def create_gateways(g): ## function to create gateways
    '''while True:
        try:
            g = int(input("Enter number of gateways: "))
        except ValueError:
            print("Invalid input, try again")
            continue
        if(g<=0):
            print("Number of gateways cannot be negative & zero ")
            continue
        else:
            break'''
    gateways = [Gateway() for i in range(g)]
    for i in range(g):
        while True:
            try:
                #power = int(input("Enter the power in gateway{}: ".format(i)))
                power = np.random.randint(0,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(power<0):
                print("Power cannot be negative ")
                continue
            else:
                break
        while True:
            try:
                #storage = int(input("Enter the storage in gateway{}: ".format(i)))
                storage = np.random.randint(0,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(storage<0):
                print("Storage cannot be negative ")
                continue
            else:
                break
        gateways[i].set_values(power,storage)
    return gateways

def create_graph(num_of_nodes,num_of_gateways):
    graph = Graph(num_of_nodes,num_of_gateways)
    for i in range(num_of_nodes):
        for j in range(num_of_gateways):
            while True:
                try:
                    #graph.set_values(i,j,int(input("Enter if there is an edge b/w node {} to gateway {}: ".format(i,j))))
                    graph.set_values(i,j,np.random.randint(0,2))
                except ValueError:
                    print("Invalid input, try again")
                    continue
                if(graph.adjmatrix[i][j]<0):
                    print("Value cannot be negative ")
                    continue
                elif(graph.adjmatrix[i][j]>1):
                    print("Value cannot be greater than 1 ")
                    continue
                else:
                    break
    return graph

def gateways_connected_to_nodes(nodes,gateways,matrix):
    gateways_connected_to_nodes_list={}
    for i in range(len(nodes)):
        for j in range(len(gateways)):
            if(matrix.adjmatrix[i][j]==1 and gateways[j].storage>0):
                if(i in gateways_connected_to_nodes_list):
                    gateways_connected_to_nodes_list[i].append(j)
                else:
                    gateways_connected_to_nodes_list[i]=[j]
    return gateways_connected_to_nodes_list

def check_storage(val,gateways):
    arr = []
    for i in val:
        if(gateways[i].get_storage()>0):
            arr.append(i)
    return arr


def TotalStorage(gateways): ## function to calculate the total power
    total_storage=0
    for i in range(len(gateways)):
        total_storage+=gateways[i].storage
    return total_storage

def TotalData(nodes): ## function to calculate the total data
    total_data=0
    for i in range(len(nodes)):
        total_data+=nodes[i].data
    return total_data

def FindCombinations(values):
    combinations = []
    for L in range(0,len(values)+1):
        for subset in itertools.combinations(values, L):
            if(len(subset)>1):
                combinations.append(list(subset))
    return combinations


def FindRatio(node_val,combinations,gateways,nodes):
    cost = {}
    ratio = float("inf")
    #print(combinations)
    for i in range(len(combinations)):
        total = 0
        #print(combinations[i])
        for j in combinations[i]:
            total += gateways[j].power
        if(nodes[node_val].data>=1): ## if data is greater than 1   
            ratio = total/nodes[node_val].data
            temp = tuple(combinations[i])
            cost[temp] = ratio
    return cost
            

def brute_force(nodes,gateways,matrix):
    final_gateways = []
    intial_storage = TotalStorage(gateways)
    intial_data = TotalData(nodes) 
    total_storage = TotalStorage(gateways)
    total_data = TotalData(nodes)
    while(total_storage>0 and total_data>0):
        gateways_connected_to_nodes_list = gateways_connected_to_nodes(nodes,gateways,matrix)
        #print(gateways_connected_to_nodes_list)
        find_all_combinations = {}
        for key, values in gateways_connected_to_nodes_list.items():
            find_all_combinations[key] = FindCombinations(values)

        #print(find_all_combinations)
        calculate_the_cost = {}
        for key, vals in find_all_combinations.items(): 
            calculate_the_cost[key] = FindRatio(key,vals,gateways,nodes)
        #print(calculate_the_cost)

        minimum = float('inf')
        for key, vals in calculate_the_cost.items():
            for key1, vals1 in vals.items():
                if(vals1<minimum):
                    minimum = vals1
                    gate_selected = key1
                    node_selected = key
        #print(node_selected,gate_selected,minimum)  

        if(minimum!=float('inf')):
            for i in gate_selected:
                if(gateways[i].get_storage()>=nodes[node_selected].data):
                    data_transferred = nodes[node_selected].data
                    gateways[i].storage -= data_transferred
                    nodes[node_selected].data = 0
                else:
                    if(nodes[node_selected].data>0):
                        data_transferred = gateways[i].storage
                        gateways[i].storage = 0
                        nodes[node_selected].data -= data_transferred
            final_gateways.append(gate_selected)
            total_storage -= data_transferred
            total_data -=  data_transferred
        else:
            #print("No more combinations")
            break
    final_storage = TotalStorage(gateways)
    final_data = TotalData(nodes)
    total_power_consumed = intial_storage - final_storage
    total_data_transferred = final_data - intial_data
    return total_power_consumed
        

'''if __name__ == "__main__":
    nodes = create_nodes()
    gateways = create_gateways()
    matrix = create_graph(len(nodes),len(gateways))
    brute_force(nodes,gateways,matrix)'''

Datalist = {}
if __name__ == '__main__':
    arr=[]
    for i in range(5,101,5):
        datalist = {}
        for j in range(50):
            nodes = create_nodes(i)

            gateways = create_gateways(15)
            graph = create_graph(len(nodes),len(gateways))
            
    
            datalist[j] =  brute_force(nodes,gateways,graph)
        df = pd.Series(datalist)
        #print(df)
        Datalist[i] = df.mean()
        print(Datalist)
    
    df = pd.Series(Datalist)
    df.to_frame()
    df.columns = ['Nodes', 'Power consumption']
    df.to_csv('/tmp/nodes.csv')
    print(df)
    print(list(df.values))
    


{5: 239.02}
{5: 239.02, 10: 456.64}
{5: 239.02, 10: 456.64, 15: 618.6}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96, 25: 701.66}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96, 25: 701.66, 30: 678.04}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96, 25: 701.66, 30: 678.04, 35: 696.96}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96, 25: 701.66, 30: 678.04, 35: 696.96, 40: 730.82}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96, 25: 701.66, 30: 678.04, 35: 696.96, 40: 730.82, 45: 721.6}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96, 25: 701.66, 30: 678.04, 35: 696.96, 40: 730.82, 45: 721.6, 50: 733.68}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96, 25: 701.66, 30: 678.04, 35: 696.96, 40: 730.82, 45: 721.6, 50: 733.68, 55: 718.42}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96, 25: 701.66, 30: 678.04, 35: 696.96, 40: 730.82, 45: 721.6, 50: 733.68, 55: 718.42, 60: 730.5}
{5: 239.02, 10: 456.64, 15: 618.6, 20: 685.96, 25: 701.66, 30: 678.04, 35: 696.

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs.layout import Title

In [ ]:
x = [x for x in range(5,101,5)]
y = [138.14, 193.76, 208.1, 212.96, 218.12, 206.14, 194.72, 214.58, 232.82, 220.9, 218.62, 209.88, 217.32, 223.66, 216.04, 195.0, 218.24, 214.36, 220.12, 225.44]
z = [229.22, 377.92, 427.86, 437.16, 461.62, 466.38, 470.62, 472.32, 496.18, 469.14, 476.5, 488.1, 469.02, 469.64, 452.4, 480.68, 458.72, 461.22, 472.96, 475.14]
m = [239.02, 456.64, 618.6, 685.96, 701.66, 678.04, 696.96, 730.82, 721.6, 733.68, 718.42, 730.5, 732.62, 737.06, 706.04, 724.22, 721.2, 724.38, 690.2, 708.68]
n = [254.16, 479.72, 712.64, 886.9, 972.12, 950.98, 961.92, 954.46, 930.74, 943.74, 955.26, 965.36, 971.74, 957.98, 956.26, 938.9, 944.58, 987.42, 939.56, 971.1]


In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=x, y=y, name="Gateways=20", line=dict(color="blue"), mode="lines+markers")   
)
fig.add_trace(
    go.Scatter(x=x, y=z, name="Gateways=40", mode='lines+markers')
)
fig.add_trace(
    go.Scatter(x=x, y=m, name="Gateways=60", mode="lines+markers")
)
fig.add_trace(
    go.Scatter(x=x, y=n, name="Gateways=80", mode="lines+markers")
)
fig.update_layout(
    title = "Total Power Consumption v/s Variable Nodes with Constant Gateways",
    xaxis_title = "Variable Nodes",
    yaxis_title = "Total Power Consumption",
    #xaxis = dict(range=[0,100]),
    #yaxis = dict(range=[300,3000])
)
fig.update_traces(textposition = "bottom right")

In [ ]:
import itertools

class Node:
    def __init__(self):
        self.data = 0
        self.battery = 0
    def set_values(self,data,battery):
        self.data = data
        self.battery = battery
    def get_data(self):
        return self.data
    def get_battery(self):
        return self.battery

class Gateway:
    def __init__(self):
        self.power = 0
        self.storage = 0
    def set_values(self,power,storage):
        self.power = power
        self.storage = storage
    def get_power(self):
        return self.power
    def get_storage(self):
        return self.storage

class Graph:	## class for creating graph objects
    def __init__(self,num_of_nodes,num_of_gateways):
        self.rows=num_of_nodes
        self.cols=num_of_gateways
        self.adjmatrix=[]
        for i in range(self.rows):
            self.adjmatrix.append([0 for j in range(self.cols)])
    def set_values(self,row,col,value):
        self.adjmatrix[row][col]=value
    def print_graph(self):
        for i in range(self.rows):
            for j in range(self.cols):
                print(self.adjmatrix[i][j],end=" ")
            print('\n')
        print('\n')

def create_nodes(n): ## function to create nodes
    '''while True:
        try:
            n = int(input("Enter number of nodes: "))
        except ValueError:
                print("Invalid input, try again")
                continue
        if n<=0:
            print("Number of nodes cannot be negative & zero ")
            continue
        else:
            break'''
    nodes = [Node() for i in range(n)]
    for i in range(n):
        #battery = int(input("Enter the battery in node{}: ".format(i)))
        #data = int(input("Enter the data in node{}: ".format(i)))
        data  = np.random.randint(1,100)
        nodes[i].set_values(data,data)
    return nodes

def create_gateways(g): ## function to create gateways
    '''while True:
        try:
            g = int(input("Enter number of gateways: "))
        except ValueError:
            print("Invalid input, try again")
            continue
        if(g<=0):
            print("Number of gateways cannot be negative & zero ")
            continue
        else:
            break'''
    gateways = [Gateway() for i in range(g)]
    for i in range(g):
        while True:
            try:
                #power = int(input("Enter the power in gateway{}: ".format(i)))
                power = np.random.randint(1,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(power<0):
                print("Power cannot be negative ")
                continue
            else:
                break
        while True:
            try:
                #storage = int(input("Enter the storage in gateway{}: ".format(i)))
                storage = np.random.randint(1,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(storage<0):
                print("Storage cannot be negative ")
                continue
            else:
                break
        gateways[i].set_values(power,storage)
    return gateways

def create_graph(num_of_nodes,num_of_gateways):
    graph = Graph(num_of_nodes,num_of_gateways)
    for i in range(num_of_nodes):
        for j in range(num_of_gateways):
            while True:
                try:
                    #graph.set_values(i,j,int(input("Enter if there is an edge b/w node {} to gateway {}: ".format(i,j))))
                    graph.set_values(i,j,np.random.randint(0,2))
                except ValueError:
                    print("Invalid input, try again")
                    continue
                if(graph.adjmatrix[i][j]<0):
                    print("Value cannot be negative ")
                    continue
                elif(graph.adjmatrix[i][j]>1):
                    print("Value cannot be greater than 1 ")
                    continue
                else:
                    break
    return graph

def gateways_connected_to_nodes(nodes,gateways,matrix):
    gateways_connected_to_nodes_list={}
    for i in range(len(nodes)):
        for j in range(len(gateways)):
            if(matrix.adjmatrix[i][j]==1 and gateways[j].storage>0):
                if(i in gateways_connected_to_nodes_list):
                    gateways_connected_to_nodes_list[i].append(j)
                else:
                    gateways_connected_to_nodes_list[i]=[j]
    return gateways_connected_to_nodes_list

def check_storage(val,gateways):
    arr = []
    for i in val:
        if(gateways[i].get_storage()>0):
            arr.append(i)
    return arr


def TotalStorage(gateways): ## function to calculate the total power
    total_storage=0
    for i in range(len(gateways)):
        total_storage+=gateways[i].storage
    return total_storage

def TotalData(nodes): ## function to calculate the total data
    total_data=0
    for i in range(len(nodes)):
        total_data+=nodes[i].data
    return total_data

def FindCombinations(values):
    combinations = []
    for L in range(0,len(values)+1):
        for subset in itertools.combinations(values, L):
            if(len(subset)>1):
                combinations.append(list(subset))
    return combinations


def FindRatio(node_val,combinations,gateways,nodes):
    cost = {}
    ratio = float("inf")
    #print(combinations)
    for i in range(len(combinations)):
        total = 0
        #print(combinations[i])
        for j in combinations[i]:
            total += gateways[j].power
        if(nodes[node_val].data>=1): ## if data is greater than 1   
            ratio = total/nodes[node_val].data
            temp = tuple(combinations[i])
            cost[temp] = ratio
    return cost
            

def brute_force(nodes,gateways,matrix):
    final_gateways = []
    intial_storage = TotalStorage(gateways)
    intial_data = TotalData(nodes) 
    total_storage = TotalStorage(gateways)
    total_data = TotalData(nodes)
    while(total_storage>0 and total_data>0):
        gateways_connected_to_nodes_list = gateways_connected_to_nodes(nodes,gateways,matrix)
        #print(gateways_connected_to_nodes_list)
        find_all_combinations = {}
        for key, values in gateways_connected_to_nodes_list.items():
            find_all_combinations[key] = FindCombinations(values)

        #print(find_all_combinations)
        calculate_the_cost = {}
        for key, vals in find_all_combinations.items(): 
            calculate_the_cost[key] = FindRatio(key,vals,gateways,nodes)
        #print(calculate_the_cost)

        minimum = float('inf')
        for key, vals in calculate_the_cost.items():
            for key1, vals1 in vals.items():
                if(vals1<minimum):
                    minimum = vals1
                    gate_selected = key1
                    node_selected = key
        #print(node_selected,gate_selected,minimum)  

        if(minimum!=float('inf')):
            for i in gate_selected:
                if(gateways[i].get_storage()>=nodes[node_selected].data):
                    data_transferred = nodes[node_selected].data
                    gateways[i].storage -= data_transferred
                    nodes[node_selected].data = 0
                else:
                    if(nodes[node_selected].data>0):
                        data_transferred = gateways[i].storage
                        gateways[i].storage = 0
                        nodes[node_selected].data -= data_transferred
            final_gateways.append(gate_selected)
            total_storage -= data_transferred
            total_data -=  data_transferred
        else:
            #print("No more combinations")
            break
    final_storage = TotalStorage(gateways)
    final_data = TotalData(nodes)
    total_power_consumed = intial_storage - final_storage
    total_data_transferred = final_data - intial_data
    return total_power_consumed
        

'''if __name__ == "__main__":
    nodes = create_nodes()
    gateways = create_gateways()
    matrix = create_graph(len(nodes),len(gateways))
    brute_force(nodes,gateways,matrix)'''

Datalist = {}
if __name__ == '__main__':
    arr=[]
    for i in range(1,21,1):
        datalist = {}
        for j in range(10):
            nodes = create_nodes(20)

            gateways = create_gateways(i)
            graph = create_graph(len(nodes),len(gateways))
            
    
            datalist[j] =  brute_force(nodes,gateways,graph)
        df = pd.Series(datalist)
        #print(df)
        Datalist[i] = df.mean()
        print(Datalist)
    
    df = pd.Series(Datalist)
    df.to_frame()
    df.columns = ['Nodes', 'Power consumption']
    df.to_csv('/tmp/nodes.csv')
    print(df)
    print(list(df.values))

{1: 0.0}
{1: 0.0, 2: 88.3}
{1: 0.0, 2: 88.3, 3: 111.7}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7, 6: 299.1}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7, 6: 299.1, 7: 296.1}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7, 6: 299.1, 7: 296.1, 8: 365.5}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7, 6: 299.1, 7: 296.1, 8: 365.5, 9: 386.6}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7, 6: 299.1, 7: 296.1, 8: 365.5, 9: 386.6, 10: 463.6}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7, 6: 299.1, 7: 296.1, 8: 365.5, 9: 386.6, 10: 463.6, 11: 486.8}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7, 6: 299.1, 7: 296.1, 8: 365.5, 9: 386.6, 10: 463.6, 11: 486.8, 12: 534.0}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7, 6: 299.1, 7: 296.1, 8: 365.5, 9: 386.6, 10: 463.6, 11: 486.8, 12: 534.0, 13: 599.2}
{1: 0.0, 2: 88.3, 3: 111.7, 4: 177.1, 5: 209.7, 6: 299.1, 7: 296.1, 8: 365.5, 9: 386.6, 10: 463.6, 11: 48

In [ ]:
x = [x for x in range(5,101,5)]
y = [0.0, 43.1, 94.1, 105.7, 116.4, 156.5, 204.8, 214.7, 173.1, 227.1, 195.0, 277.4, 225.6, 274.9, 237.6, 271.7, 257.5, 226.7, 222.8, 248.5]
z = [0.0, 74.7, 93.9, 161.7, 157.8, 228.7, 301.9, 334.8, 349.1, 378.5, 433.1, 430.1, 484.6, 495.3, 493.1, 510.8, 492.5, 475.5, 493.7, 534.8]
m = [0.0, 70.9, 128.8, 178.8, 220.8, 257.9, 291.8, 358.9, 416.4, 444.3, 489.7, 516.5, 574.9, 605.7, 624.3, 696.1, 664.9, 707.9, 672.1, 711.7]
n = [0.0, 88.3, 111.7, 177.1, 209.7, 299.1, 296.1, 365.5, 386.6, 463.6, 486.8, 534.0, 599.2, 624.4, 721.7, 708.8, 719.7, 792.5, 922.5, 896.2]

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs.layout import Title

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=x, y=y, name="Nodes=20", mode="lines+markers")    
)

fig.add_trace(
    go.Scatter(x=x, y=z ,name="Nodes=40", mode="lines+markers")
)

fig.add_trace(
    go.Scatter(x=x, y=m, name="Nodes=60", mode="lines+markers")
)

fig.add_trace(
    go.Scatter(x=x, y=n, name="Nodes=80", mode="lines+markers")
)
fig.update_layout(
    title = "Total Power Consumption v/s Variable Gateways with Constant Nodes",
    xaxis_title = "Variable Gateways",
    yaxis_title = "Total Power Consumption",
    #xaxis = dict(range=[0,100]),
    #yaxis = dict(range=[300,3000])

)
fig.update_traces(textposition = "bottom right")


fig.show()

In [13]:
import itertools

class Node:
    def __init__(self):
        self.data = 0
        self.battery = 0
    def set_values(self,data,battery):
        self.data = data
        self.battery = battery
    def get_data(self):
        return self.data
    def get_battery(self):
        return self.battery

class Gateway:
    def __init__(self):
        self.power = 0
        self.storage = 0
    def set_values(self,power,storage):
        self.power = power
        self.storage = storage
    def get_power(self):
        return self.power
    def get_storage(self):
        return self.storage

class Graph:	## class for creating graph objects
    def __init__(self,num_of_nodes,num_of_gateways):
        self.rows=num_of_nodes
        self.cols=num_of_gateways
        self.adjmatrix=[]
        for i in range(self.rows):
            self.adjmatrix.append([0 for j in range(self.cols)])
    def set_values(self,row,col,value):
        self.adjmatrix[row][col]=value
    def print_graph(self):
        for i in range(self.rows):
            for j in range(self.cols):
                print(self.adjmatrix[i][j],end=" ")
            print('\n')
        print('\n')

def create_nodes(n): ## function to create nodes
    '''while True:
        try:
            n = int(input("Enter number of nodes: "))
        except ValueError:
                print("Invalid input, try again")
                continue
        if n<=0:
            print("Number of nodes cannot be negative & zero ")
            continue
        else:
            break'''
    nodes = [Node() for i in range(n)]
    for i in range(n):
        #battery = int(input("Enter the battery in node{}: ".format(i)))
        #data = int(input("Enter the data in node{}: ".format(i)))
        #data  = np.random.randint(1,100)
        data = 80
        nodes[i].set_values(data,data)
    return nodes

def create_gateways(): ## function to create gateways
    while True:
        try:
            #g = int(input("Enter number of gateways: "))
            g=5
        except ValueError:
            print("Invalid input, try again")
            continue
        if(g<=0):
            print("Number of gateways cannot be negative & zero ")
            continue
        else:
            break
    gateways = [Gateway() for i in range(g)]
    for i in range(g):
        while True:
            try:
                #power = int(input("Enter the power in gateway{}: ".format(i)))
                power = np.random.randint(1,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(power<0):
                print("Power cannot be negative ")
                continue
            else:
                break
        while True:
            try:
                #storage = int(input("Enter the storage in gateway{}: ".format(i)))
                storage = np.random.randint(1,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(storage<0):
                print("Storage cannot be negative ")
                continue
            else:
                break
        gateways[i].set_values(power,storage)
    return gateways

def create_graph(num_of_nodes,num_of_gateways):
    graph = Graph(num_of_nodes,num_of_gateways)
    for i in range(num_of_nodes):
        for j in range(num_of_gateways):
            while True:
                try:
                    #graph.set_values(i,j,int(input("Enter if there is an edge b/w node {} to gateway {}: ".format(i,j))))
                    graph.set_values(i,j,np.random.randint(0,2))
                except ValueError:
                    print("Invalid input, try again")
                    continue
                if(graph.adjmatrix[i][j]<0):
                    print("Value cannot be negative ")
                    continue
                elif(graph.adjmatrix[i][j]>1):
                    print("Value cannot be greater than 1 ")
                    continue
                else:
                    break
    return graph

def gateways_connected_to_nodes(nodes,gateways,matrix):
    gateways_connected_to_nodes_list={}
    for i in range(len(nodes)):
        for j in range(len(gateways)):
            if(matrix.adjmatrix[i][j]==1 and gateways[j].storage>0):
                if(i in gateways_connected_to_nodes_list):
                    gateways_connected_to_nodes_list[i].append(j)
                else:
                    gateways_connected_to_nodes_list[i]=[j]
    return gateways_connected_to_nodes_list

def check_storage(val,gateways):
    arr = []
    for i in val:
        if(gateways[i].get_storage()>0):
            arr.append(i)
    return arr


def TotalStorage(gateways): ## function to calculate the total power
    total_storage=0
    for i in range(len(gateways)):
        total_storage+=gateways[i].storage
    return total_storage

def TotalData(nodes): ## function to calculate the total data
    total_data=0
    for i in range(len(nodes)):
        total_data+=nodes[i].data
    return total_data

def FindCombinations(values):
    combinations = []
    for L in range(0,len(values)+1):
        for subset in itertools.combinations(values, L):
            if(len(subset)>1):
                combinations.append(list(subset))
    return combinations


def FindRatio(node_val,combinations,gateways,nodes):
    cost = {}
    ratio = float("inf")
    #print(combinations)
    for i in range(len(combinations)):
        total = 0
        #print(combinations[i])
        for j in combinations[i]:
            total += gateways[j].power
        if(nodes[node_val].data>=1): ## if data is greater than 1   
            ratio = total/nodes[node_val].data
            temp = tuple(combinations[i])
            cost[temp] = ratio
    return cost
            

def brute_force(nodes,gateways,matrix):
    final_gateways = []
    intial_storage = TotalStorage(gateways)
    intial_data = TotalData(nodes) 
    total_storage = TotalStorage(gateways)
    total_data = TotalData(nodes)
    while(total_storage>0 and total_data>0):
        gateways_connected_to_nodes_list = gateways_connected_to_nodes(nodes,gateways,matrix)
        #print(gateways_connected_to_nodes_list)
        find_all_combinations = {}
        for key, values in gateways_connected_to_nodes_list.items():
            find_all_combinations[key] = FindCombinations(values)

        #print(find_all_combinations)
        calculate_the_cost = {}
        for key, vals in find_all_combinations.items(): 
            calculate_the_cost[key] = FindRatio(key,vals,gateways,nodes)
        #print(calculate_the_cost)

        minimum = float('inf')
        for key, vals in calculate_the_cost.items():
            for key1, vals1 in vals.items():
                if(vals1<minimum):
                    minimum = vals1
                    gate_selected = key1
                    node_selected = key
        #print(node_selected,gate_selected,minimum)  

        if(minimum!=float('inf')):
            for i in gate_selected:
                if(gateways[i].get_storage()>=nodes[node_selected].data):
                    data_transferred = nodes[node_selected].data
                    gateways[i].storage -= data_transferred
                    nodes[node_selected].data = 0
                else:
                    if(nodes[node_selected].data>0):
                        data_transferred = gateways[i].storage
                        gateways[i].storage = 0
                        nodes[node_selected].data -= data_transferred
            final_gateways.append(gate_selected)
            total_storage -= data_transferred
            total_data -=  data_transferred
        else:
            #print("No more combinations")
            break
    final_storage = TotalStorage(gateways)
    final_data = TotalData(nodes)
    total_power_consumed = intial_storage - final_storage
    total_data_transferred = final_data - intial_data
    return total_power_consumed
        

'''if __name__ == "__main__":
    nodes = create_nodes()
    gateways = create_gateways()
    matrix = create_graph(len(nodes),len(gateways))
    brute_force(nodes,gateways,matrix)'''

Datalist = {}
if __name__ == '__main__':
    arr=[]
    for i in range(1,21,1):
        datalist = {}
        for j in range(10):
            nodes = create_nodes(i)

            gateways = create_gateways()
            graph = create_graph(len(nodes),len(gateways))
            
    
            datalist[j] =  brute_force(nodes,gateways,graph)
        df = pd.Series(datalist)
        #print(df)
        Datalist[i] = df.mean()
        print(Datalist)
    
    df = pd.Series(Datalist)
    df.to_frame()
    df.columns = ['Nodes', 'Power consumption']
    df.to_csv('/tmp/nodes.csv')
    print(df)
    print(list(df.values))

{1: 63.1}
{1: 63.1, 2: 113.3}
{1: 63.1, 2: 113.3, 3: 173.5}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9, 6: 189.3}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9, 6: 189.3, 7: 231.3}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9, 6: 189.3, 7: 231.3, 8: 200.0}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9, 6: 189.3, 7: 231.3, 8: 200.0, 9: 216.4}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9, 6: 189.3, 7: 231.3, 8: 200.0, 9: 216.4, 10: 245.2}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9, 6: 189.3, 7: 231.3, 8: 200.0, 9: 216.4, 10: 245.2, 11: 203.7}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9, 6: 189.3, 7: 231.3, 8: 200.0, 9: 216.4, 10: 245.2, 11: 203.7, 12: 206.4}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9, 6: 189.3, 7: 231.3, 8: 200.0, 9: 216.4, 10: 245.2, 11: 203.7, 12: 206.4, 13: 207.2}
{1: 63.1, 2: 113.3, 3: 173.5, 4: 164.5, 5: 172.9, 6: 189.3, 7: 231.3, 8: 200.0, 

In [ ]:
x = x = [x for x in range(5,101,5)]
y = [40.0, 80.0, 120.0, 160.0, 200.0, 240.0, 280.0, 320.0, 360.0]
z = [80.0, 160.0, 240.0, 320.0, 400.0, 480.0, 555.8, 632.0, 692.0]
m = [120.0, 240.0, 360.0, 479.6, 586.3, 706.5, 803.6, 857.9, 895.9]
n = [160.0, 320.0, 472.0, 631.5, 762.7, 872.7, 901.6, 967.8, 979.5]
y1 = [63.1, 113.3, 173.5, 164.5, 172.9, 189.3, 231.3, 200.0, 216.4, 245.2, 203.7, 206.4, 207.2, 216.1, 206.5, 242.2, 165.8, 227.1, 184.0, 210.6]
z1 = [78.2, 142.1, 226.8, 300.0, 382.4, 402.7, 410.3, 393.8, 426.4, 461.1, 416.2, 450.8, 471.5, 470.4, 422.9, 476.9, 479.3, 470.3, 440.7, 434.5]
m1 = [80.0, 160.0, 238.0, 308.0, 391.7, 455.0, 537.0, 564.0, 675.7, 602.2, 686.8, 701.4, 653.9, 684.2, 715.8, 698.6, 732.7, 760.0, 733.1, 694.0]
n1 = [80.0, 160.0, 240.0, 320.0, 400.0, 480.0, 553.7, 637.2, 716.1, 754.3, 798.6, 846.6, 830.7, 910.2, 954.7, 977.4, 978.5, 951.9, 983.7, 936.0]




In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=x, y=y, name="data=20", mode="lines+markers")    
)

fig.add_trace(
    go.Scatter(x=x, y=z ,name="data=40", mode="lines+markers")
)

fig.add_trace(
    go.Scatter(x=x, y=m, name="data=60", mode="lines+markers")
)

fig.add_trace(
    go.Scatter(x=x, y=n, name="data=80", mode="lines+markers")
)
fig.update_layout(
    title = "Total Power Consumption v/s Variable Nodes and data is constant with Constant Gateways i,e 20",
    xaxis_title = "Variable Nodes",
    yaxis_title = "Total Power Consumption",
    #xaxis = dict(range=[0,100]),
    #yaxis = dict(range=[300,3000])

)
fig.update_traces(textposition = "bottom right")


fig.show()



In [ ]:
import itertools

class Node:
    def __init__(self):
        self.data = 0
        self.battery = 0
    def set_values(self,data,battery):
        self.data = data
        self.battery = battery
    def get_data(self):
        return self.data
    def get_battery(self):
        return self.battery

class Gateway:
    def __init__(self):
        self.power = 0
        self.storage = 0
    def set_values(self,power,storage):
        self.power = power
        self.storage = storage
    def get_power(self):
        return self.power
    def get_storage(self):
        return self.storage

class Graph:	## class for creating graph objects
    def __init__(self,num_of_nodes,num_of_gateways):
        self.rows=num_of_nodes
        self.cols=num_of_gateways
        self.adjmatrix=[]
        for i in range(self.rows):
            self.adjmatrix.append([0 for j in range(self.cols)])
    def set_values(self,row,col,value):
        self.adjmatrix[row][col]=value
    def print_graph(self):
        for i in range(self.rows):
            for j in range(self.cols):
                print(self.adjmatrix[i][j],end=" ")
            print('\n')
        print('\n')

def create_nodes(n): ## function to create nodes
    '''while True:
        try:
            n = int(input("Enter number of nodes: "))
        except ValueError:
                print("Invalid input, try again")
                continue
        if n<=0:
            print("Number of nodes cannot be negative & zero ")
            continue
        else:
            break'''
    nodes = [Node() for i in range(n)]
    for i in range(n):
        #battery = int(input("Enter the battery in node{}: ".format(i)))
        #data = int(input("Enter the data in node{}: ".format(i)))
        data  = np.random.randint(1,100)
        nodes[i].set_values(data,data)
    return nodes

def create_gateways(): ## function to create gateways
    while True:
        try:
            #g = int(input("Enter number of gateways: "))
            g=22
        except ValueError:
            print("Invalid input, try again")
            continue
        if(g<=0):
            print("Number of gateways cannot be negative & zero ")
            continue
        else:
            break
    gateways = [Gateway() for i in range(g)]
    for i in range(g):
        while True:
            try:
                #power = int(input("Enter the power in gateway{}: ".format(i)))
                power = np.random.randint(1,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(power<0):
                print("Power cannot be negative ")
                continue
            else:
                break
        while True:
            try:
                #storage = int(input("Enter the storage in gateway{}: ".format(i)))
                storage = np.random.randint(1,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(storage<0):
                print("Storage cannot be negative ")
                continue
            else:
                break
        gateways[i].set_values(power,storage)
    return gateways

def create_graph(num_of_nodes,num_of_gateways):
    graph = Graph(num_of_nodes,num_of_gateways)
    for i in range(num_of_nodes):
        for j in range(num_of_gateways):
            while True:
                try:
                    #graph.set_values(i,j,int(input("Enter if there is an edge b/w node {} to gateway {}: ".format(i,j))))
                    graph.set_values(i,j,np.random.randint(0,2))
                except ValueError:
                    print("Invalid input, try again")
                    continue
                if(graph.adjmatrix[i][j]<0):
                    print("Value cannot be negative ")
                    continue
                elif(graph.adjmatrix[i][j]>1):
                    print("Value cannot be greater than 1 ")
                    continue
                else:
                    break
    return graph

def gateways_connected_to_nodes(nodes,gateways,matrix):
    gateways_connected_to_nodes_list={}
    for i in range(len(nodes)):
        for j in range(len(gateways)):
            if(matrix.adjmatrix[i][j]==1 and gateways[j].storage>0):
                if(i in gateways_connected_to_nodes_list):
                    gateways_connected_to_nodes_list[i].append(j)
                else:
                    gateways_connected_to_nodes_list[i]=[j]
    return gateways_connected_to_nodes_list

def check_storage(val,gateways):
    arr = []
    for i in val:
        if(gateways[i].get_storage()>0):
            arr.append(i)
    return arr


def TotalStorage(gateways): ## function to calculate the total power
    total_storage=0
    for i in range(len(gateways)):
        total_storage+=gateways[i].storage
    return total_storage

def TotalData(nodes): ## function to calculate the total data
    total_data=0
    for i in range(len(nodes)):
        total_data+=nodes[i].data
    return total_data

def FindCombinations(values):
    combinations = []
    for L in range(0,len(values)+1):
        for subset in itertools.combinations(values, L):
            if(len(subset)>1):
                combinations.append(list(subset))
    return combinations


def FindRatio(node_val,combinations,gateways,nodes):
    cost = {}
    ratio = float("inf")
    #print(combinations)
    for i in range(len(combinations)):
        total = 0
        #print(combinations[i])
        for j in combinations[i]:
            total += gateways[j].power
        if(nodes[node_val].data>=1): ## if data is greater than 1   
            ratio = total/nodes[node_val].data
            temp = tuple(combinations[i])
            cost[temp] = ratio
    return cost
            

def brute_force(nodes,gateways,matrix):
    final_gateways = []
    intial_storage = TotalStorage(gateways)
    intial_data = TotalData(nodes) 
    total_storage = TotalStorage(gateways)
    total_data = TotalData(nodes)
    count=0
    while(total_storage>0 and total_data>0):
        gateways_connected_to_nodes_list = gateways_connected_to_nodes(nodes,gateways,matrix)
        #print(gateways_connected_to_nodes_list)
        find_all_combinations = {}
        for key, values in gateways_connected_to_nodes_list.items():
            find_all_combinations[key] = FindCombinations(values)

        #print(find_all_combinations)
        calculate_the_cost = {}
        for key, vals in find_all_combinations.items(): 
            calculate_the_cost[key] = FindRatio(key,vals,gateways,nodes)
        #print(calculate_the_cost)

        minimum = float('inf')
        for key, vals in calculate_the_cost.items():
            for key1, vals1 in vals.items():
                if(vals1<minimum):
                    minimum = vals1
                    gate_selected = key1
                    node_selected = key
        #print(node_selected,gate_selected,minimum)  

        if(minimum!=float('inf')):
            for i in gate_selected:
                if(gateways[i].get_storage()>=nodes[node_selected].data):
                    data_transferred = nodes[node_selected].data
                    gateways[i].storage -= data_transferred
                    nodes[node_selected].data = 0
                else:
                    if(nodes[node_selected].data>0):
                        data_transferred = gateways[i].storage
                        gateways[i].storage = 0
                        nodes[node_selected].data -= data_transferred
            final_gateways.append(gate_selected)
            total_storage -= data_transferred
            total_data -=  data_transferred
        else:
            #print("No more combinations")
            break
    '''final_storage = TotalStorage(gateways)
    final_data = TotalData(nodes)
    total_power_consumed = intial_storage - final_storage
    total_data_transferred = final_data - intial_data
    return total_power_consumed'''
    for node in nodes:
      if node.data<=0:
        count+=1
    return count 
        

'''if __name__ == "__main__":
    nodes = create_nodes()
    gateways = create_gateways()
    matrix = create_graph(len(nodes),len(gateways))
    brute_force(nodes,gateways,matrix)'''

Datalist = {}
if __name__ == '__main__':
    arr=[]
    for i in range(5,101,5):
        datalist = {}
        for j in range(100):
            nodes = create_nodes(i)

            gateways = create_gateways()
            graph = create_graph(len(nodes),len(gateways))
            
    
            datalist[j] =  brute_force(nodes,gateways,graph)
        df = pd.Series(datalist)
        #print(df)
        Datalist[i] = df.mean()
        print(Datalist)
    
    df = pd.Series(Datalist)
    print(list(df.values))

{5: 5.0}
{5: 5.0, 10: 9.96}
{5: 5.0, 10: 9.96, 15: 14.31}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47, 30: 11.98}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47, 30: 11.98, 35: 10.6}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47, 30: 11.98, 35: 10.6, 40: 9.86}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47, 30: 11.98, 35: 10.6, 40: 9.86, 45: 9.18}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47, 30: 11.98, 35: 10.6, 40: 9.86, 45: 9.18, 50: 8.7}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47, 30: 11.98, 35: 10.6, 40: 9.86, 45: 9.18, 50: 8.7, 55: 8.18}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47, 30: 11.98, 35: 10.6, 40: 9.86, 45: 9.18, 50: 8.7, 55: 8.18, 60: 8.04}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47, 30: 11.98, 35: 10.6, 40: 9.86, 45: 9.18, 50: 8.7, 55: 8.18, 60: 8.04, 65: 8.02}
{5: 5.0, 10: 9.96, 15: 14.31, 20: 16.71, 25: 14.47, 30: 11.98, 35: 10

KeyboardInterrupt: ignored

In [ ]:
x = x = x = [x for x in range(5,101,5)]
y = [2.0, 3.4, 4.7, 6.0, 6.4, 6.3, 6.1, 5.7, 4.7, 4.8]
z = [2.0, 3.8, 5.9, 7.3, 8.9, 10.5, 9.6, 10.3, 9.7, 8.4]
m = [2.0, 4.0, 6.0, 8.0, 10.0, 11.9, 13.8, 14.5, 15.5, 12.7]
n = [2.0, 4.0, 6.0, 8.0, 9.9, 12.0, 13.8, 15.6, 17.3, 18.6]


In [ ]:
x = [x for x in range(5,101,5)]
y = [4.29, 6.12, 6.2, 4.72, 3.95, 4.03, 3.82, 3.4, 3.43, 3.16, 3.32, 3.18, 2.94, 3.18, 3.09, 3.08, 2.85, 3.08, 2.86, 2.87]
z = [4.86, 8.98, 9.63, 9.04, 8.02, 6.35, 5.91, 5.8, 5.48, 5.3, 5.19, 5.02, 5.03, 4.9, 4.68, 4.71, 4.57, 4.66, 4.39, 4.6]
m = [4.99, 9.81, 13.34, 14.82, 12.95, 10.46, 9.79, 8.79, 8.31, 7.73, 7.49, 6.69, 7.04, 6.91, 6.8, 6.55, 6.36, 6.46, 6.22, 6.37]
n = [5.0, 9.96, 14.31, 16.71, 14.47, 11.98, 10.6, 9.86, 9.18, 8.7, 8.18, 8.04, 8.02, 7.73, 7.51, 7.38, 7.21, 6.91, 6.8, 6.6]

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=x, y=y, name="Gateways=20", mode="lines+markers")    
)
fig.add_trace(
    go.Scatter(x=x, y=z, name="Gateways=40", mode="lines+markers")    
)
fig.add_trace(
    go.Scatter(x=x, y=m, name="Gateways=60", mode="lines+markers")    
)
fig.add_trace(
    
    go.Scatter(x=x, y=n, name="Gateways=80", mode="lines+markers")    
)
fig.update_layout(
    title = "Data sent Fully v/s Variable Nodes  with Constant Gateways ",
    xaxis_title = "Variable Nodes",
    yaxis_title = "Nodes Fully Sent Data",
    #xaxis = dict(range=[0,100]),
    #yaxis = dict(range=[300,3000])

)
fig.update_traces(textposition = "bottom right")


fig.show()

In [ ]:
import itertools
import numpy as np
import pandas as pd


class Node:
    def __init__(self):
        self.data = 0
        self.battery = 0
    def set_values(self,data,battery):
        self.data = data
        self.battery = battery
    def get_data(self):
        return self.data
    def get_battery(self):
        return self.battery

class Gateway:
    def __init__(self):
        self.power = 0
        self.storage = 0
    def set_values(self,power,storage):
        self.power = power
        self.storage = storage
    def get_power(self):
        return self.power
    def get_storage(self):
        return self.storage

class Graph:	## class for creating graph objects
    def __init__(self,num_of_nodes,num_of_gateways):
        self.rows=num_of_nodes
        self.cols=num_of_gateways
        self.adjmatrix=[]
        for i in range(self.rows):
            self.adjmatrix.append([0 for j in range(self.cols)])
    def set_values(self,row,col,value):
        self.adjmatrix[row][col]=value
    def print_graph(self):
        for i in range(self.rows):
            for j in range(self.cols):
                print(self.adjmatrix[i][j],end=" ")
            print('\n')
        print('\n')

def create_nodes(n): ## function to create nodes
    '''while True:
        try:
            n = int(input("Enter number of nodes: "))
        except ValueError:
                print("Invalid input, try again")
                continue
        if n<=0:
            print("Number of nodes cannot be negative & zero ")
            continue
        else:
            break'''
    nodes = [Node() for i in range(n)]
    for i in range(n):
        #battery = int(input("Enter the battery in node{}: ".format(i)))
        #data = int(input("Enter the data in node{}: ".format(i)))
        data  = np.random.randint(1,100)
        nodes[i].set_values(data,data)
    return nodes

def create_gateways(i): ## function to create gateways
    while True:
        try:
            #g = int(input("Enter number of gateways: "))
            g = i
        except ValueError:
            print("Invalid input, try again")
            continue
        if(g<=0):
            print("Number of gateways cannot be negative & zero ")
            continue
        else:
            break
    gateways = [Gateway() for i in range(g)]
    for i in range(g):
        while True:
            try:
                #power = int(input("Enter the power in gateway{}: ".format(i)))
                power = np.random.randint(1,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(power<0):
                print("Power cannot be negative ")
                continue
            else:
                break
        while True:
            try:
                #storage = int(input("Enter the storage in gateway{}: ".format(i)))
                storage = np.random.randint(1,100)
            except ValueError:
                print("Invalid input, try again")
                continue
            if(storage<0):
                print("Storage cannot be negative ")
                continue
            else:
                break
        gateways[i].set_values(power,storage)
    return gateways

def create_graph(num_of_nodes,num_of_gateways):
    graph = Graph(num_of_nodes,num_of_gateways)
    for i in range(num_of_nodes):
        for j in range(num_of_gateways):
            while True:
                try:
                    #graph.set_values(i,j,int(input("Enter if there is an edge b/w node {} to gateway {}: ".format(i,j))))
                    graph.set_values(i,j,np.random.randint(0,2))
                except ValueError:
                    print("Invalid input, try again")
                    continue
                if(graph.adjmatrix[i][j]<0):
                    print("Value cannot be negative ")
                    continue
                elif(graph.adjmatrix[i][j]>1):
                    print("Value cannot be greater than 1 ")
                    continue
                else:
                    break
    return graph

def gateways_connected_to_nodes(nodes,gateways,matrix):
    gateways_connected_to_nodes_list={}
    for i in range(len(nodes)):
        for j in range(len(gateways)):
            if(matrix.adjmatrix[i][j]==1 and gateways[j].storage>0):
                if(i in gateways_connected_to_nodes_list):
                    gateways_connected_to_nodes_list[i].append(j)
                else:
                    gateways_connected_to_nodes_list[i]=[j]
    return gateways_connected_to_nodes_list

def check_storage(val,gateways):
    arr = []
    for i in val:
        if(gateways[i].get_storage()>0):
            arr.append(i)
    return arr


def TotalStorage(gateways): ## function to calculate the total power
    total_storage=0
    for i in range(len(gateways)):
        total_storage+=gateways[i].storage
    return total_storage

def TotalData(nodes): ## function to calculate the total data
    total_data=0
    for i in range(len(nodes)):
        total_data+=nodes[i].data
    return total_data

def FindCombinations(values):
    combinations = []
    for L in range(0,len(values)+1):
        for subset in itertools.combinations(values, L):
            if(len(subset)>1):
                combinations.append(list(subset))
    return combinations


def FindRatio(node_val,combinations,gateways,nodes):
    cost = {}
    ratio = float("inf")
    #print(combinations)
    for i in range(len(combinations)):
        total = 0
        #print(combinations[i])
        for j in combinations[i]:
            total += gateways[j].power
        if(nodes[node_val].data>=1): ## if data is greater than 1   
            ratio = total/nodes[node_val].data
            temp = tuple(combinations[i])
            cost[temp] = ratio
    return cost
            

def brute_force(nodes,gateways,matrix):
    final_gateways = []
    intial_storage = TotalStorage(gateways)
    intial_data = TotalData(nodes) 
    total_storage = TotalStorage(gateways)
    total_data = TotalData(nodes)
    count=0
    while(total_storage>0 and total_data>0):
        gateways_connected_to_nodes_list = gateways_connected_to_nodes(nodes,gateways,matrix)
        #print(gateways_connected_to_nodes_list)
        find_all_combinations = {}
        for key, values in gateways_connected_to_nodes_list.items():
            find_all_combinations[key] = FindCombinations(values)

        #print(find_all_combinations)
        calculate_the_cost = {}
        for key, vals in find_all_combinations.items(): 
            calculate_the_cost[key] = FindRatio(key,vals,gateways,nodes)
        #print(calculate_the_cost)

        minimum = float('inf')
        for key, vals in calculate_the_cost.items():
            for key1, vals1 in vals.items():
                if(vals1<minimum):
                    minimum = vals1
                    gate_selected = key1
                    node_selected = key
        #print(node_selected,gate_selected,minimum)  

        if(minimum!=float('inf')):
            for i in gate_selected:
                if(gateways[i].get_storage()>=nodes[node_selected].data):
                    data_transferred = nodes[node_selected].data
                    gateways[i].storage -= data_transferred
                    nodes[node_selected].data = 0
                else:
                    if(nodes[node_selected].data>0):
                        data_transferred = gateways[i].storage
                        gateways[i].storage = 0
                        nodes[node_selected].data -= data_transferred
            final_gateways.append(gate_selected)
            total_storage -= data_transferred
            total_data -=  data_transferred
        else:
            #print("No more combinations")
            break
    '''final_storage = TotalStorage(gateways)
    final_data = TotalData(nodes)
    total_power_consumed = intial_storage - final_storage
    total_data_transferred = final_data - intial_data
    return total_power_consumed'''
    for node in nodes:
      if node.data<=0:
        count+=1
    return count 
        

'''if __name__ == "__main__":
    nodes = create_nodes()
    gateways = create_gateways()
    matrix = create_graph(len(nodes),len(gateways))
    brute_force(nodes,gateways,matrix)'''

Datalist = {}
if __name__ == '__main__':
    arr=[]
    for i in range(2,22,2):
        datalist = {}
        for j in range(50):
            nodes = create_nodes(300)

            gateways = create_gateways(i)
            graph = create_graph(len(nodes),len(gateways))
            
    
            datalist[j] =  brute_force(nodes,gateways,graph)
        df = pd.Series(datalist)
        #print(df)
        Datalist[i] = df.mean()
        print(Datalist)
    
    df = pd.Series(Datalist)
    print(list(df.values))

{2: 0.68}
{2: 0.68, 4: 1.04}
{2: 0.68, 4: 1.04, 6: 1.74}
{2: 0.68, 4: 1.04, 6: 1.74, 8: 1.96}
{2: 0.68, 4: 1.04, 6: 1.74, 8: 1.96, 10: 2.62}
{2: 0.68, 4: 1.04, 6: 1.74, 8: 1.96, 10: 2.62, 12: 3.52}
{2: 0.68, 4: 1.04, 6: 1.74, 8: 1.96, 10: 2.62, 12: 3.52, 14: 3.76}
{2: 0.68, 4: 1.04, 6: 1.74, 8: 1.96, 10: 2.62, 12: 3.52, 14: 3.76, 16: 4.46}
{2: 0.68, 4: 1.04, 6: 1.74, 8: 1.96, 10: 2.62, 12: 3.52, 14: 3.76, 16: 4.46, 18: 4.9}
{2: 0.68, 4: 1.04, 6: 1.74, 8: 1.96, 10: 2.62, 12: 3.52, 14: 3.76, 16: 4.46, 18: 4.9, 20: 5.54}
[0.68, 1.04, 1.74, 1.96, 2.62, 3.52, 3.76, 4.46, 4.9, 5.54]


In [ ]:
x = [x for x in range(5,100,5)]
y = [1.3, 1.1, 2.9, 4.2, 5.9, 8.3, 8.8, 8.8, 9.8, 10.0]
z = [0.6, 1.9, 1.9, 2.7, 4.0, 6.7, 8.4, 10.8, 11.4, 13.1]
m = [0.7, 1.7, 2.1, 4.0, 3.3, 5.5, 6.6, 8.7, 8.6, 10.5]
n = [0.5, 0.9, 2.2, 2.7, 3.9, 4.4, 5.5, 5.7, 8.2, 7.5]

In [ ]:
x = [x for x in range(5,100,5)]
y = [0.56, 1.22, 1.58, 2.52, 2.92, 3.46, 4.52, 4.54, 5.42, 6.38]
z = [0.52, 1.0, 1.54, 2.24, 2.82, 3.42, 4.52, 4.28, 5.22, 5.52]
m = 

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs.layout import Title

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=x, y=y, name="Nodes=10", mode="lines+markers")    
)
fig.add_trace(
    go.Scatter(x=x, y=z, name="Nodes=20", mode="lines+markers")    
)
fig.add_trace(
    go.Scatter(x=x, y=m, name="Nodes=30", mode="lines+markers")    
)
fig.add_trace(
    go.Scatter(x=x, y=n, name="Nodes=40", mode="lines+markers")    
)
fig.update_layout(
    title = "Total Power Consumption v/s Variable Gateways  and Constant Nodes for optimized",
    xaxis_title = "Gateways",
    yaxis_title = "Total Power Consumption",
    #xaxis = dict(range=[0,100]),
    #yaxis = dict(range=[300,3000])

)
fig.update_traces(textposition = "bottom right")
fig.show()